<a href="https://colab.research.google.com/github/m-wessler/nbm-verification/blob/main/CoLab_Familiarization_Metadata_Pull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cartopy

In [ ]:
import json
import requests
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
token = '' #@param {type:'string'}
cwa = '' #@param {type:'string'}
network = 'select network' #@param ['select network', 'NWS', 'NWS+RAWS', 'NWS+RAWS+HADS', 'NWS', 'RAWS', 'HADS', 'ALL'] {type:'string'}
variable = 'select variable' #@param ['select variable', 'air_temp' ,'precip', 'humidity', 'wind', 'gust', 'snow', 'cloud'] {type:'string'}

In [ ]:
metadata_api = 'https://api.synopticdata.com/v2/stations/metadata?'

# Convert named network to Synoptic API numeric network IDs (see docs)
synoptic_networks = {
    "NWS+RAWS+HADS":"1,2,106",
    "NWS+RAWS":"1,2",
    "NWS":"1",
    "RAWS": "2",
    "HADS":"106",
    "ALL":None}

network_query = (f"&network={synoptic_networks[network]}"
                 if synoptic_networks[network] is not None else '')

# Assemble the API query
api_query = (f"{metadata_api}&token={token}&cwa={cwa}" + network_query +
            f"&complete=1&sensorvars=1,obrange=20230118")

# Print the API query to output
print(api_query)

# Get the data from the API
response = requests.get(api_query)
metadata = pd.DataFrame(response.json()['STATION'])

# Remove NaNs and index by network, station ID
metadata = metadata[metadata['MNET_SHORTNAME'].notna()]
metadata = metadata.set_index(['MNET_SHORTNAME', 'STID'])

metadata['LATITUDE'] = metadata['LATITUDE'].astype(float)
metadata['LONGITUDE'] = metadata['LONGITUDE'].astype(float)
metadata['ELEVATION'] = metadata['ELEVATION'].astype(float)

metadata = metadata[metadata['LATITUDE'] >= 31]
metadata = metadata[metadata['LONGITUDE'] <= -103.00]
metadata = metadata[metadata['STATUS'] == 'ACTIVE']

variable_mask = np.array([i for i, md in enumerate(metadata['SENSOR_VARIABLES'])
                                if variable in str(md.keys())])

metadata = metadata.iloc[variable_mask]

print(metadata.shape)
metadata.head()

In [ ]:
network_names = metadata.index.get_level_values(0).unique()

print('Station Counts:')
for nn in network_names:
    print(nn, metadata.loc[nn, :].shape[0])

In [ ]:
print('Station Elevations:')

fig, axs = plt.subplots(network_names.size, 1, sharex=True,
                        figsize=(8, 1.5*network_names.size))
axs = axs.flatten()

for i, nn in enumerate(network_names):
    axs[i].hist(metadata.loc[nn, :]['ELEVATION'],
             bins=np.arange(0, np.ceil(metadata.ELEVATION.max()/1000)*1000, 500),
                edgecolor='k', alpha=0.5, width=475,
             label=f'{nn} ({metadata.loc[nn, :].shape[0]})')
    axs[i].grid(zorder=-1)
    axs[i].set_xlim(left=0)
    axs[i].legend()

axs[i].set_xlabel('Elevation (ft)')

fig.subplots_adjust(hspace=0.1)
plt.show()

In [ ]:
import geopandas as gpd
import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from cartopy.io.img_tiles import GoogleTiles
from mpl_toolkits.axes_grid1 import make_axes_locatable

geo_metadata = gpd.GeoDataFrame(metadata,
                                geometry=gpd.points_from_xy(
                                    metadata.LONGITUDE, metadata.LATITUDE),
                                crs="EPSG:4326")

cmap = mpl.cm.terrain
vmin = np.floor(metadata.ELEVATION.min()/1000)*1000
vmax = np.ceil(metadata.ELEVATION.max()/1000)*1000
norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

markers = [",","o","v","^","<",">",
           "1","2","3","4","8","s","p","P","*","h",
           "H","+","x","X","D","d","|","_",
           0,1,2,3,4,5,6,7,8,9,10,11,
           ",","o","v","^","<",">",
           "1","2","3","4","8","s","p","P","*","h",
           "H","+","x","X","D","d","|","_",
           0,1,2,3,4,5,6,7,8,9,10,11]

# Create a figure and axes object
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

for i, nn in enumerate(network_names):
    geo_metadata.loc[nn, :].plot(column='ELEVATION', ax=ax, zorder=10,
                    markersize=75, marker=markers[i], label=nn,
                    vmin=vmin, vmax=vmax, cmap=cmap, norm=norm)


# # Add state boundaries
ax.add_feature(cfeature.LAND.with_scale('50m'))
ax.add_feature(cfeature.OCEAN.with_scale('50m'))
ax.add_feature(cfeature.LAKES.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'))
ax.coastlines()

if network_names.size <= 5:
    plt.legend()

# cb = ax.get_children()[2]
plt.colorbar(cb, ax=ax, shrink=0.5, label='Elevation (ft)')

plt.title(f'{cwa.upper()} Stations ({network})')

plt.show()